In [ ]:
from importlib import reload
import pandas as pd

from risk_lib.data import get_portfolios

from risk_lib.corr_mds import *


In [ ]:
from importlib import reload
import risk_lib.corr_mds as corr_mds
reload(corr_mds)
from risk_lib.corr_mds import *

In [ ]:
# TODO: Enlarge hyperfactors, either with a separate trace or with a size parameter
# TODO: Add portfolios of factors
# TODO: Add ret/vol scatter, in the MDS format, with 45-degree line, maybe whiskers
# TODO: Color whiskers as markers

# TODO: Rename ETF to PCW

# TODO: For t_0, rotate so SPX is on x-axis. For t_1, 

Collect data

In [ ]:
from risk_lib.data import get_factor_data
factor_data = get_factor_data(halflifes=[21, 63, 126, 252])
factor_master = pd.DataFrame(factor_data.factor_name.attrs).T
factor_data


Project factor correlation matrix onto two dimensions, using different linear transformations to align the data.

In [ ]:
transformation_type = {None:        'no transformation', 
                       'rotate':    'SPX rotated to x-axis', 
                       'normalize': 'SPX transformed to (1, 0)'
                       }

for transformation in transformation_type.keys():
    mds_ts = (mds_ts_df(factor_data.corr, transformation=transformation, start_date='2024')
              .reset_index()
              .join(factor_master, on='factor_name')
              .assign(date = lambda df: df['date'].astype(str)))
    fig = (draw_mds_ts(mds_ts, tick_range=None)
           .update_layout(title=f'Correlation Projection ({transformation_type[transformation]})'))
    fig.show()
    fig.write_html(R'G:\My Drive\risk\correlation.html')


Instead of animation, here we use whiskers.

In [ ]:
# mds_latest['hyper_factor'] += 1 
# mds_latest['hyper_factor'] *= 10
# mds_latest['hyper_factor'] = pd.to_numeric(mds_latest['hyper_factor'])


In [ ]:
mds_ts['size'].astype('int')

In [ ]:
t0 = '2024-11-08'
t1 = '2024-10-09'
t2 = '2024-08-08'

mds_ts = (mds_ts_df(factor_data.corr, transformation='normalize', start_date='2024')
            .reset_index()
            .join(factor_master, on='asset')
            .assign(date = lambda df: df['date'].astype(str))
            .assign(size = lambda df: df['hyper_factor'].mul(1).add(.5).astype('float'))
            )
mds_latest = mds_ts[mds_ts['date'] == t0].drop(columns='date')
fig = draw_mds_ts(mds_latest, tick_range=1.5)
fig = add_whiskers(fig, mds_ts, t0, t1)
fig = add_whiskers(fig, mds_ts, t1, t2)
fig.update_layout(legend_title_text=None)

mds_previous = mds_ts[mds_ts['date'] == t1].drop(columns='date')
fig_previous = (draw_mds_ts(mds_previous, tick_range=1.5)
                .update_traces(text=None,  opacity=0.3) # marker_size=5,
                .update_layout(showlegend=False))
for trace in fig_previous.data:
    trace.showlegend = False
    fig.add_trace(trace)

# for trace in fig.data:
#     if trace.mode == 'lines':
#         for marker_trace in fig.data:
#             if marker_trace.mode == 'markers+text' and marker_trace.name == trace.legendgroup:
#                 trace.update(line=dict(color=marker_trace.marker.color))
#                 break

fig.show(renderer='png')


In [ ]:
date_max = mds_ts['date'].max()
mds_latest = mds_ts[mds_ts['date'] == date_max].drop(columns=['date'])
mds_latest['hyper_factor'] = mds_latest['hyper_factor'].astype(int)
fig = draw_mds_ts(mds_latest, tick_range=1.5)
fig.show(renderer='png')

fig2 = px.scatter(mds_latest, x='dim1', y='dim2', color='asset_class', text='asset', size='hyper_factor', size_max=10)

# # Add line traces connecting corresponding points
# for i in range(len(df_time_1)):
#     fig.add_trace(go.Scatter(x=[df_time_1['x'][i], df_time_2['x'][i]], y=[df_time_1['y'][i], df_time_2['y'][i]], mode='lines', line=dict(color='gray', width=1), showlegend=False))


In [ ]:
mds_latest

In [ ]:
mds_latest['hyper_factor'] = mds_latest['hyper_factor'].astype(int)

In [ ]:
px.scatter(mds_latest, x='dim1', y='dim2', color='asset_class', text='asset', size='hyper_factor', size_max=10)


In [ ]:
factor_data.date.max()

In [ ]:
# Vol chart

In [ ]:
from risk_lib.util import xr_pct_change

date_latest = pd.to_datetime(factor_data.date.max().item())
date_latest = '2024-11-11'
vol_type = 63

vol = factor_data.vol.sel(date=date_latest, vol_type=vol_type).to_series()
# ret = factor_data.ret.sel(date=date_latest).to_series()
ret = xr_pct_change(factor_data.cret, dim='date', periods=21).sel(date=date_latest).to_series().rename('ret').mul(100)

df = pd.concat([ret, vol, factor_master], axis=1).reset_index()


fig_format = {'template': 'plotly_white', 'height': 750, 'width': 750}
fig = (px.scatter(df, x='vol', y='ret', color='asset_class', text='index', **fig_format)
       .update_traces(textposition='middle right', 
                      textfont_color='lightgray')
       .update_layout(# xaxis_title=None,
                        #yaxis_title=None,
                        # xaxis_showticklabels=False,
                        # yaxis_showticklabels=False,
                    #   xaxis_showgrid=False,
                    #   yaxis_showgrid=False,
                      xaxis_showline=False,
                      yaxis_showline=False,
                       legend_title_text=None,
                       )
       )



fig.show(renderer='png')


# PORTFOLIOS

In [ ]:
from risk_lib.stats import get_volatility_set, get_correlation_set

halflifes = [21, 63,] #126, 252, 512]
portfolios = get_portfolios().reindex(factor_master.index).fillna(0)
factor_ret = factor_data.ret.to_pandas()
portfolio_ret = factor_ret @ portfolios
combined_ret = pd.concat([factor_ret, portfolio_ret], axis=1).rename_axis(index='date', columns='asset')

ds = xr.Dataset()
ds['ret'] = combined_ret
ds['vol']   = get_volatility_set(ds['ret'], halflifes)
ds['corr']  = get_correlation_set(ds['ret'], halflifes)
ds['asset'].attrs = factor_master.T

portfolio_data = ds

In [ ]:
portfolio_data

t0 = '2024-11-08'
t1 = '2024-10-09'
t2 = '2024-08-08'

mds_ts = (mds_ts_df(portfolio_data.corr, transformation='normalize', start_date='2024')
            .reset_index()
            .join(factor_master, on='asset')
            .assign(date = lambda df: df['date'].astype(str))
            .assign(size = lambda df: df['hyper_factor'].mul(1).add(.5).astype('float'))
            )
mds_latest = mds_ts[mds_ts['date'] == t0].drop(columns='date')
fig = draw_mds_ts(mds_latest, tick_range=1.5)
fig = add_whiskers(fig, mds_ts, t0, t1)
fig = add_whiskers(fig, mds_ts, t1, t2)
fig.update_layout(legend_title_text=None)

mds_previous = mds_ts[mds_ts['date'] == t1].drop(columns='date')
fig_previous = (draw_mds_ts(mds_previous, tick_range=1.5)
                .update_traces(text=None,  opacity=0.3) # marker_size=5,
                .update_layout(showlegend=False))
for trace in fig_previous.data:
    trace.showlegend = False
    fig.add_trace(trace)

# for trace in fig.data:
#     if trace.mode == 'lines':
#         for marker_trace in fig.data:
#             if marker_trace.mode == 'markers+text' and marker_trace.name == trace.legendgroup:
#                 trace.update(line=dict(color=marker_trace.marker.color))
#                 break

fig.show(renderer='png')


In [ ]:
factor_master = d.get_factor_master(sheet_name='read2')
asset_list = factor_master[factor_master['composite'] != 1].index.to_list()

ds = xr.Dataset()





In [ ]:
# Test

In [ ]:
mds_test= (mds_ts_df(portfolio_data.corr, transformation=None, start_date='2024')
            .reset_index()
            .join(factor_master, on='asset')
            .assign(date = lambda df: df['date'].astype(str))
            .assign(size = lambda df: df['hyper_factor'].mul(1).add(.5).astype('float'))
            )

In [ ]:
assets = ['SPY', 'TLT']
date = '2024-11-14'
mds_test.query("asset in @assets and date==@date")

x0, y0 = mds_test.set_index(['date', 'asset']).loc[('2024-11-14', 'SPY'), ['dim1', 'dim2']].values
x1, y1 = mds_test.set_index(['date', 'asset']).loc[('2024-11-14', 'TLT'), ['dim1', 'dim2']].values

sqrt((x1 - x0)**2 + (y1 - y0)**2)

In [ ]:
corr = portfolio_data.corr.sel(date='2024-11-14', corr_type=21).to_pandas()
sqrt(1-corr**2).loc['SPY', 'TLT']

In [ ]:
mds_test.set_index(['date', 'asset'])[['dim1', 'dim2']].to_xarray()
mds_test.sel(asset='SPY')

In [ ]:
import numpy as np

# Parameters
n = 1000  # Number of points

# Radius computation
# Sum of distances for points on a circle
def average_pairwise_distance_circle(n, R):
    total_distance = 0
    for k in range(1, n):  # Pairwise angular separation
        angular_distance = np.pi * k / n  # Angle in radians
        total_distance += (n - k) * 2 * R * np.sin(angular_distance)
    return total_distance / (n * (n - 1) / 2)  # Normalize by number of pairs

# Target average distance
target_distance = 1

# Find the radius numerically
R_values = np.linspace(0.001, 100, 1000)  # Test radius values
average_distances = [average_pairwise_distance_circle(n, R) for R in R_values]

# Find the closest match to the target distance
best_R_index = np.argmin(np.abs(np.array(average_distances) - target_distance))
best_R = R_values[best_R_index]

best_R